# Validation of TPM and VBR models against each other with DP SMIB Load Step

## Load Libraries

In [ ]:
import re
import numpy as np
import math
import os
import subprocess
import pickle
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt

#%matplotlib widget

## Simulation parameters

In [ ]:
root_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
path_exec = root_path + '/build/dpsim/examples/cxx/'

name_executable_vbr = 'DP_SMIB_ReducedOrderSG_LoadStep'
name_vbr = "DP_SMIB_ReducedOrderSG_VBR_LoadStep"

name_executable_tpm = 'DP_SMIB_ReducedOrderSGIterative_LoadStep'
name_tpm = "DP_SMIB_ReducedOrderSGIterative_LoadStep"

# times in s
timestep = 1e-3
load_step_time = 10.0
roi_begin = 10.0
roi_end = 12.0

# tpm config params
max_iter_array = [0, 1, 2, 5, 10, 20]
tolerance = 1e-10
sg_model = '4TPM'


roi_begin_idx = int(roi_begin/timestep)
roi_end_idx = int(roi_end/timestep)

timestep_str = '{:1.6f}'.format(timestep)

logs_path = 'logs'
if not os.path.exists(logs_path):
    os.mkdir(logs_path)
    
var_name = 'SynGen.Te'

te_ref = 0.5454986888690558

## VBR Simulations

In [ ]:
options_list_vbr = []
options_list_vbr.append('loadStepEventTime=' + str(load_step_time))
options_list_vbr.append('TimeStep=' + timestep_str)
options_list_vbr.append('SimName=' + name_vbr)

args_options_list_vbr = []
for option in options_list_vbr:
    args_options_list_vbr.extend(['--option', option])

simVBR = subprocess.Popen([path_exec + name_executable_vbr] + args_options_list_vbr, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(simVBR.communicate()[0].decode())

In [ ]:
results_path_vbr = logs_path + "/" + name_vbr + "/" + name_vbr + ".csv"
ts_vbr = read_timeseries_dpsim(results_path_vbr)[var_name]
ts_vbr_roi = TimeSeries(ts_vbr.name+'roi',ts_vbr.time[roi_begin_idx:roi_end_idx],ts_vbr.values[roi_begin_idx:roi_end_idx])

## TPM Simulations (with different maximum iterations)

In [ ]:
options_list_tpm = []
options_list_tpm.append('loadStepEventTime=' + str(load_step_time))
options_list_tpm.append('TimeStep=' + timestep_str)
options_list_tpm.append('Tolerance=' + str(tolerance))
options_list_tpm.append('SGModel=' + str(sg_model))  

args_options_list_tpm = []
for option in options_list_tpm:
    args_options_list_tpm.extend(['--option', option])

for max_iter in max_iter_array:
    name_iter = name_tpm + '_MaxIter' + str(max_iter)
    simTPM = subprocess.Popen([path_exec + name_executable_tpm, '--option', 'SimName=' + name_iter, '--option', 'MaxIter=' + str(max_iter)] + args_options_list_tpm, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print(simTPM.communicate()[0].decode())

In [ ]:
ts_tpm = []
ts_tpm_roi = []
for max_iter in max_iter_array:
    name_iter = name_tpm + '_MaxIter' + str(max_iter)
    results_path_iter = logs_path + "/" + name_iter + "/" + name_iter + ".csv"
    ts_tpm.append(read_timeseries_dpsim(results_path_iter)[var_name])
    ts_tpm_roi.append(TimeSeries(ts_tpm[-1].name+'roi',ts_tpm[-1].time[roi_begin_idx:roi_end_idx],ts_tpm[-1].values[roi_begin_idx:roi_end_idx]))

## Comparative Plots

### Complete

In [ ]:
plt.figure()
plt.plot(ts_vbr.time, ts_vbr.values, label='VBR')

for max_iter in max_iter_array:
    max_iter_idx = max_iter_array.index(max_iter)
    plt.plot(ts_tpm[max_iter_idx].time, ts_tpm[max_iter_idx].values, linestyle='--', label='TPM - MaxIter' + str(max_iter))

plt.legend()
plt.show()

### ROI

In [ ]:
plt.figure()
plt.plot(ts_vbr_roi.time, ts_vbr_roi.values, label='VBR')

for max_iter in max_iter_array:
    max_iter_idx = max_iter_array.index(max_iter)
    plt.plot(ts_tpm_roi[max_iter_idx].time, ts_tpm_roi[max_iter_idx].values, linestyle='--', label='TPM - MaxIter' + str(max_iter))

plt.legend()
plt.show()

## RMSE Calculation

In [ ]:
rmse_list = []
for max_iter in max_iter_array:
    max_iter_idx = max_iter_array.index(max_iter)
    rmse_list.append(ts_vbr_roi.rmse(ts_tpm_roi[max_iter_idx], ts_vbr_roi)/te_ref*100)
    print('RMSE of TPM with MaxIter={}: {}%'.format(max_iter,rmse_list[-1]))

## Assertions

In [ ]:
assert(rmse_list[0]<0.32)
assert(rmse_list[1]<0.54)
assert(rmse_list[2]<0.31)
assert(rmse_list[3]<0.07)
assert(rmse_list[4]<0.003)
assert(rmse_list[5]<2.4e-5)